<a href="https://colab.research.google.com/github/d0r6y/AI-Project/blob/main/TextBugger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Textbugger**

In [ ]:
!pip install textattack

     |████████████████████████████████| 337kB 9.7MB/s 
     |████████████████████████████████| 1.1MB 13.4MB/s 
     |████████████████████████████████| 778kB 23.7MB/s 
     |████████████████████████████████| 337kB 44.4MB/s 
     |████████████████████████████████| 163kB 39.5MB/s 
     |████████████████████████████████| 20.1MB 1.4MB/s 
     |████████████████████████████████| 61kB 10.2MB/s 
     |████████████████████████████████| 3.0MB 49.8MB/s 
     |████████████████████████████████| 102kB 15.6MB/s 
     |████████████████████████████████| 1.1MB 42.4MB/s 
     |████████████████████████████████| 890kB 48.1MB/s 
     |████████████████████████████████| 19.7MB 42.2MB/s 
     |████████████████████████████████| 71kB 11.1MB/s 
     |████████████████████████████████| 983kB 49.3MB/s 
     |████████████████████████████████| 798kB 49.5MB/s 
     |████████████████████████████████| 17.7MB 218kB/s 
     |████████████████████████████████| 245kB 55.6MB/s 
  Created wheel for terminaltables: filename=termi

In [ ]:
# Import the model
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb")
tokenizer = AutoTokenizer("textattack/bert-base-uncased-imdb")

# Import the dataset
from textattack.datasets import HuggingFaceDataset
dataset = HuggingFaceDataset("imdb", None, "test")

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Loading datasets dataset imdb, split test.


Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3. Subsequent calls will reuse this data.


In [ ]:
"""
TextBugger
===============
(TextBugger: Generating Adversarial Text Against Real-world Applications)
"""

from textattack.constraints.pre_transformation import (
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.shared.attack import Attack
from textattack.transformations import (
    CompositeTransformation,
    WordSwapEmbedding,
    WordSwapHomoglyphSwap,
    WordSwapNeighboringCharacterSwap,
    WordSwapRandomCharacterDeletion,
    WordSwapRandomCharacterInsertion,
    WordSwapChangeName,
    WordSwapRandomCharacterSubstitution,
      
)



In [ ]:
constraints = [RepeatModification(), StopwordModification()]
       
constraints.append(UniversalSentenceEncoder(threshold=0.8))

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

goal_function = UntargetedClassification(model_wrapper)
 
search_method = GreedyWordSwapWIR(wir_method="delete")

resolver HttpCompressedFileResolver does not support the provided handle.
resolver GcsCompressedFileResolver does not support the provided handle.
textattack: Goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'> compatible with model BertForSequenceClassification.


In [ ]:
transformation = CompositeTransformation(
            [
                WordSwapRandomCharacterSubstitution()
            ]
        )
            

In [ ]:
attack = Attack(goal_function, constraints, transformation, search_method)
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapRandomCharacterSubstitution(
        (random_one):  True
      )
    )
  (constraints): 
    (0): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
)


In [ ]:
from textattack.loggers import CSVLogger # tracks a dataframe for us.
from textattack.attack_results import SuccessfulAttackResult
from textattack.loggers import AttackLogManager
import time

start_time = time.time()

results_iterable = attack.attack_dataset(dataset)

logger = CSVLogger(color_method='html')

num_successes = 0
while num_successes < 20:
    result = next(results_iterable)
    if isinstance(result, SuccessfulAttackResult):
        logger.log_attack_result(result)
        num_successes += 1
        print(f'{num_successes} of 20 successes complete.')

finish_time = time.time()
logger.flush()
print(finish_time-start_time)

1 of 20 successes complete.
2 of 20 successes complete.
3 of 20 successes complete.
4 of 20 successes complete.
5 of 20 successes complete.
6 of 20 successes complete.
7 of 20 successes complete.
8 of 20 successes complete.
9 of 20 successes complete.
10 of 20 successes complete.
11 of 20 successes complete.
12 of 20 successes complete.
13 of 20 successes complete.
14 of 20 successes complete.
15 of 20 successes complete.
16 of 20 successes complete.
17 of 20 successes complete.
18 of 20 successes complete.
19 of 20 successes complete.
20 of 20 successes complete.
224.3024823665619


In [ ]:
logger.flush()
import pandas as pd
pd.options.display.max_colwidth = 480 # increase colum width so we can actually read the examples

from IPython.core.display import display, HTML
display(HTML(logger.df[['original_text', 'perturbed_text']].to_html(escape=False)))